In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/MyDrive/Code\ Repository/aidea/ANS/AIdea_ANS

/gdrive/MyDrive/Code Repository/aidea/ANS/AIdea_ANS


In [ ]:
!pip install -r requirements.txt

In [ ]:
# %%
from email.mime import audio
import enum
from pickletools import optimize
import yaml
import torch
import tqdm
from math import floor

from model.dataset import AudioDataset
from model.models import Generator, Discriminator, Model
from model.utils import custom_pesq, show_plt, pesq, save_flac

with open("config.yaml") as fp:
    cfg = yaml.load(fp, Loader=yaml.FullLoader)

print(cfg)
dataset = AudioDataset(cfg['train_data_path'], cfg['class'])

train_num = floor(dataset.__len__() * cfg['train_per'])
print(train_num)
train_set, val_set = torch.utils.data.random_split(
    dataset, [train_num, dataset.__len__() - train_num])

dataloader = torch.utils.data.DataLoader(
    dataset=train_set, batch_size=cfg.get('batch_size'), shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device: ', device)

{'train_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/train_classification', 'test_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/test', 'saving_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data', 'class': 'train', 'train_per': 0.5, 'epochs': 1, 'batch_size': 32, 'lr': 0.0002, 'beta1': 0.5, 'beta2': 0.999}
832
device:  cpu


In [ ]:
net = Model().to(device)


netG_X2Y = Generator().to(device)
netG_Y2X = Generator().to(device)
netD_X = Discriminator().to(device)
netD_Y = Discriminator().to(device)


loss_func = torch.nn.MSELoss().to(device)
# %%


g_params = list(netG_X2Y.parameters()) + list(netG_Y2X.parameters())  
# Get generator parameters

optimizer = torch.optim.Adam(net.parameters(), lr=cfg['lr'], betas=(cfg['beta1'], cfg['beta2']))

# Create optimizers for the generators and discriminators
g_optimizer = torch.optim.Adam(g_params, lr=cfg['lr'], betas=(cfg['beta1'], cfg['beta2']))

d_x_optimizer = torch.optim.Adam(
    netD_X.parameters(), lr=cfg['lr'], betas=(cfg['beta1'], cfg['beta2']))
d_y_optimizer = torch.optim.Adam(
    netD_Y.parameters(), lr=cfg['lr'], betas=(cfg['beta1'], cfg['beta2']))

loss_lst = []
pesq_lst = []

In [ ]:
def train():
    for epoch in range(cfg['epochs']):
        print('epoch: ', epoch)
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for i, data in enumerate(progress_bar):

            audio_data = data['audio'].to(device)
            rate = data['data'].to(device)

            g_optimizer.zero_grad()

            noise_pre = net(audio_data)
            audio_pre = audio_data - noise_pre
            pesq_soc = pesq(rate, audio_data, audio_pre)

            pesq_weight = custom_pesq(pesq_soc)

            loss = loss_func(audio_data, noise_pre) * pesq_weight
            loss.backward()

            pesq_lst.append(pesq_soc)
            loss_lst.append(loss)

            optimizer.step()
            # audio_X2Y = netG_X2Y(audio_data)
            # audio_Y2X = netG_Y2X(audio_data)

            # loss_X2Y = loss_func(audio_data, audio_X2Y)
            # loss_Y2X = loss_func(audio_data, audio_Y2X)

        show_plt('loss', loss_lst)
        show_plt('pesq', pesq_lst)
        save_flac(cfg['saving_path'], str(epoch) + '.flac', audio_pre, rate)

# %%


In [ ]:
train()

epoch:  0


TypeError: ignored